In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from scipy.stats import t
import cufflinks as cf
from math import log, sqrt
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import datetime as dt

from CointTests import *

In [2]:
dfq = pd.read_excel('cross_section.xlsx', parse_dates=True, index_col=0)

In [4]:
dfq.iplot()

In [5]:
#TODO: more comments here needed
def get_pairs_EG(df):
    cointegrated_portfolios = []
    adf_critical = tableADF(len(df), 0)[1]
    st_critical = t.ppf(0.05, len(df)-2)
    pairs = df.columns
    for i, c1 in enumerate(pairs):
        for c2 in pairs[i+1:]:
            EG1 = EngleGranger(df[c1], df[c2])
            EG1.fit()
            EG2 = EngleGranger(df[c2], df[c1])
            EG2.fit()
            EG = None
            if (EG1.ADFstat < adf_critical) and (EG1.ecm_tstat < st_critical):
                if (EG2.ADFstat < adf_critical) and (EG2.ecm_tstat < EG1.ecm_tstat):
                    EG = EG2
                    portfolio = (c2, c1)
                else:
                    EG = EG1
                    portfolio = (c1, c2)
            elif (EG2.ADFstat < adf_critical) and (EG2.ecm_tstat < st_critical): #checking which relationship is more significant
                EG = EG2
                portfolio = (c2, c1)
            if EG != None:
                res = np.array(EG.residuals).reshape(-1, 1)
                AR_regression = LinearRegression(res[:-1], res[1:], add_constant=False)
                AR_regression.fit()
                theta = 1 - float(AR_regression.coeff[0])
                hl  = log(2) / theta
                norm = sum(np.abs([1, -EG.coeff[1]]))
                cointegrated_portfolios.append([list(portfolio), [1, -EG.coeff[1]] / norm, hl, EG.coeff[0] / norm, EG.residuals.std() / norm])

    return pd.DataFrame(cointegrated_portfolios, columns=['Portfolio', 'Weights', 'Half-Life', 'Mean', 'Std'])

def get_triplets_J(df):
    cointegrated_portfolios = []
    pairs = df.columns
    for i, c1 in enumerate(pairs):
        for j, c2 in enumerate(pairs[i+1:]):
            for c3 in pairs[i+j+2:]:
                data = df[[c1, c2, c3]]
                res = coint_johansen(data, det_order=1, k_ar_diff=1)
                for d in range(3):
                    if (res.lr2[d] > res.cvm[d, 1]) and (res.lr1[d] > res.cvt[d, 1]):
                        norm = sum(abs(res.evec[:, d]))
                        vec_norm = res.evec[:, d] / norm
                        portfolio_value = np.array(data.dot(vec_norm)).reshape(-1, 1)
                        AR_regression = LinearRegression(portfolio_value[:-1], portfolio_value[1:])
                        AR_regression.fit()
                        theta = 1 - float(AR_regression.coeff[1])
                        hl  = log(2) / theta
                        cointegrated_portfolios.append([[c1, c2, c3], list(vec_norm), hl, portfolio_value.mean(), portfolio_value.std()])
                    else:
                        break
    return pd.DataFrame(cointegrated_portfolios, columns=['Portfolio', 'Weights', 'Half-Life', 'Mean', 'Std'])

def get_quadruplets_J(df):
    cointegrated_portfolios = []
    pairs = df.columns
    for i, c1 in enumerate(pairs):
        for j, c2 in enumerate(pairs[i+1:]):
            for k, c3 in enumerate(pairs[i+j+2:]):
                for c4 in pairs[i+j+k+3:]:
                    data = df[[c1, c2, c3, c4]] 
                    res = coint_johansen(data, det_order=1, k_ar_diff=1)
                    for d in range(3):
                        if (res.lr2[d] > res.cvm[d, 1]) and (res.lr1[d] > res.cvt[d, 1]):
                            norm = sum(abs(res.evec[:, d]))
                            vec_norm = res.evec[:, d] / norm
                            portfolio_value = np.array(data.dot(vec_norm)).reshape(-1, 1)
                            AR_regression = LinearRegression(portfolio_value[:-1], portfolio_value[1:])
                            AR_regression.fit()
                            theta = 1 - float(AR_regression.coeff[1])
                            hl  = log(2) / theta
                            cointegrated_portfolios.append([[c1, c2, c3, c4], vec_norm, hl, portfolio_value.mean(), portfolio_value.std()])
                        else:
                            break
    return pd.DataFrame(cointegrated_portfolios, columns=['Portfolio', 'Weights', 'Half-Life', 'Mean', 'Std'])

In [6]:
def select_strategies(strategies, min_weight=0.1, max_gross_weight=10, max_hl=60, min_std=0.3):
    strategies = strategies[strategies['Std'] > min_std]
    strategies = strategies[strategies['Half-Life'] < max_hl]
    for i in strategies.index:
        w = strategies.loc[i, 'Weights']
        if sum(np.abs(w)) > max_gross_weight:
            strategies = strategies.drop(i)
        elif min(np.abs(w)) < min_weight:
            strategies = strategies.drop(i)
    return strategies

def calculate_value_series(strategies, market_data):
    value_series = pd.DataFrame(index=market_data.index, columns=strategies.index)
    for i in strategies.index:
        p, w = strategies.loc[i, 'Portfolio'], strategies.loc[i, 'Weights']
        value_series[i] = market_data.loc[:,p].dot(w)
    return value_series

def calculate_z_scores(strategies, value_series):
    z_scores = pd.DataFrame(index=value_series.index, columns=strategies.index)
    for s in strategies.index:
        z_scores[s] = (value_series[s] - strategies.loc[s, 'Mean']) / strategies.loc[s, 'Std'] 
    return z_scores

def get_position(z_scores, ol=-2, cl=-1, os=2, cs=1):
    position = pd.DataFrame(0, index=z_scores.index, columns=z_scores.columns)
    position.iloc[0] = np.where(z_scores.iloc[0] <= ol, 1, np.where(z_scores.iloc[0] >= os, -1, 0))
    for i in range(1, len(z_scores.index)):
        position.iloc[i] = np.where(z_scores.iloc[i] <= ol, 1, np.where(z_scores.iloc[i] >= os, -1,
            np.where((z_scores.iloc[i] < cl) & (position.iloc[i-1] == 1), 1,
            np.where((z_scores.iloc[i] > cs) & (position.iloc[i-1] == -1), -1, 0))))
    return position

def get_trades(position):
    trades = position.diff()
    trades.iloc[0] = position.iloc[0]
    trades.iloc[-1] = -position.iloc[-2]
    return trades

In [12]:
tc = 0 #minus half bid/offer (on average)
eoy = pd.date_range(end=dfq.index[-1], start = dfq.index[0], freq='BY')
performance = pd.Series(dtype='float64')
costs = pd.Series(dtype='float64')
for i in range(1, len(eoy), 1):
    idx = pd.bdate_range(start=eoy[i-1] + pd.tseries.offsets.DateOffset(days=1), end=eoy[i])
    train_idx = idx.intersection(dfq.index)
    idx = pd.bdate_range(end=eoy[i] + pd.tseries.offsets.DateOffset(years=1), start=eoy[i] + pd.tseries.offsets.DateOffset(days=1))
    test_idx = idx.intersection(dfq.index)
    strategies = get_pairs_EG(dfq.loc[train_idx])
    #strategies = strategies.append(get_triplets_J(dfq.loc[train_idx]))
    #strategies = strategies.append(get_quadruplets_J(dfq.loc[train_idx])).reset_index(drop=True)
    #strategies = get_triplets_J(df1y_clean.loc[train_idx])
    filtered_strategies = select_strategies(strategies, min_std=0.05)
    cash_value = calculate_value_series(filtered_strategies, dfq.loc[test_idx]) 
    zsc = calculate_z_scores(filtered_strategies, cash_value)
    position = get_position(zsc, ol=-2, cl=-1, os=2, cs=1)
    #position = position.div(position.abs().sum(axis=1), axis=0)
    trades = get_trades(position)
    costs = costs.append((trades.abs() * tc).sum(axis=1))
    net_performance = performance + costs
    performance = performance.append((2 * cash_value.diff().shift(-1) * position).sum(axis=1)) # 2* here because we use net vega but strategies normilized to gross vega=1
    
dv01_pts = pd.concat([performance.cumsum(), (performance + costs).cumsum()], axis=1)
dv01_pts.columns = ['gross', 'net']
dv01_pts.iplot(title='Strategy performance in dv01 points', xTitle='Date', yTitle='DV01 points')

In [9]:
strategies

,Portfolio,Weights,Half-Life,Mean,Std
0,"[SA, HUN]","[0.7808371845907627, -0.21916281540923735]",44.523972,4.103213,0.424319
1,"[SA, MEX]","[0.5836896101119003, -0.41631038988809965]",31.757454,0.915155,0.223560
